In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

repo_link = "https://raw.githubusercontent.com/berwynwong/econ4488/master/"
year_list = [1970, 1980, 1991, 2000]
ipums_list = []

for i, year in enumerate(year_list):
    ipums_raw = pd.read_csv(repo_link+"ipums{}.csv".format(year))
    var_dict = pd.read_csv(repo_link+"var_dict", index_col=0).to_dict() # renaming some commonly used variables
    ipums_list.append(ipums_raw.rename(columns=var_dict['newvar']))

### We only consider Peninsular Malaysia male citizens who are wage workers

In [2]:
for i, df in enumerate(ipums_list):
    df = df[df.sex != 2] # Female
    df = df[df.geo1 !=458012] # Sabah, Labuan Federal Territory
    df = df[df.geo2 !=458013] # Sarawak
    df = df[df.empstat == 1] # Not employed
    df = df[df.age >=15]
    df = df[df.claswk !=3] # Unpaid
    ipums_list[i] = df

df_list = []

for i, df in enumerate(ipums_list):
    df = df[df.occ != 000]
    df = df[df.occ != 999]
    df = df[df.own != 0]
    df = df[df.birstate != 15] # Labuan
    df = df[df.marst !=0]
    df = df[df.ethmy != 99] # NIU (1991:14411, 2000:23399)
    df = df[df.empstat != 0] # NIU (54601, 48289, 89729, 99921)
    df = df[df.indgen != 000] # NIU (118484, 118426, 225137, 272021)
    df = df[df.claswk != 0] # NIU (120986, 118426, 210914, 271975)
    df_list.append(df)
    print("Shape is " + str(df.shape))
    print(df['YEAR'][0])

Shape is (31433, 36)
1970
Shape is (39378, 36)
1980
Shape is (68662, 36)
1991
Shape is (89270, 33)
2000


In [3]:
occ_missing = {1970: {998: np.NaN, 999: np.NaN},
               1980: {997: np.NaN, 998: np.NaN},
               1991: {995: np.NaN, 998: np.NaN},
               2000: {998: np.NaN, 999: np.NaN}}

missing_dict = {'age': {999: np.NaN},
                'birctry': {69900: np.NaN, 99999: np.NaN},
                'birstate': {99: np.NaN},
                'claswk': {9: np.NaN},
                'edat': {9: np.NaN},
                'edumy': {999: np.NaN},
                'empstat': {9: np.NaN},
                'ethmy': {98: np.NaN},
                'indgen': {999: np.NaN},
                'marst': {0: np.NaN, 9: np.NaN},
                'nat': {9: np.NaN},
                'own': {9: np.NaN},
                'relg': {9: np.NaN},
                'sex': {9: np.NaN},
                'urb': {9: np.NaN}}

In [4]:
for i, df in enumerate(df_list):
    df_year = df['YEAR'][0]
    for var in list(missing_dict):
        df = df.replace(missing_dict)
    df['occ'] = df['occ'].replace(occ_missing[df_year])
    df_list[i] = df

In [5]:
def df_recode(df):
    for precode, newcode in zip(['ethmy','edumy'],['race', 'edu']):
        code_dict = pd.read_csv(repo_link+newcode, index_col=0).to_dict()
        # I created dictionaries that classify Malaysian races and education in a way that makes sense given the context.
        # Categorised races into Malay, Chinese, Indian, non-Malay indigenous, Other
        # Categorised education into less than primary, primary, lower secondary, upper secondary, tertiary
        # See github repository for more details
        df[newcode] = df[precode].map(code_dict[precode])
    # I also used dictionaries for assigning ISEI scores to each occupational code
    df_year = df['YEAR'][0]
    isei_dict = pd.read_csv(repo_link+'isei'+str(df_year), index_col=0).to_dict()
    df['isei'] = df['occ'].map(isei_dict['occ'])
    return df

In [6]:
%%time
for i, df in enumerate(df_list):
    df_list[i] = df_recode(df)

Wall time: 3.31 s


In [7]:
for i, df in enumerate(df_list):
    for col in ['urb', 'geo1', 'geo2', 'own', 'nat', 'birctry',
                'birstate', 'relg', 'marst', 'empstat', 'indgen', 'claswk', 'race']:
        df[col] = df[col].astype('category')
        df_list[i]=df
        
for i, df in enumerate(df_list):
    df['edu'] = df['edu'].astype('category', ordered=True)
    df_list[i]=df

In [8]:
df_dict = {1970:df_list[0],
      1980:df_list[1],
      1991:df_list[2],
      2000:df_list[3]}

## Regression modelling

In [9]:
reg_form0 = "race"
reg_form1 = "race + edu + age + np.power(age, 2)"
reg_form2 = "race + edu + age + np.power(age, 2) + indgen + own + marst"
reg_form3 = "edu + age + np.power(age, 2) + indgen + own + marst + urb + claswk + geo2 + birstate"
reg_form4 = "race + edu + age + np.power(age, 2) + indgen + own + marst + urb + claswk + geo2 + birstate + race*edu"
reg_form5 = "race + edu + age + np.power(age, 2) + indgen + own + marst + urb + claswk + geo2 + birstate + race*edu + race*indgen+race*urb"

In [10]:
class Oaxaca(object):
        
    def __init__(self, year, races=[0,1], 
                 varlist=['edu', 'indgen', 'own', 'marst', 'urb', 'claswk', 'geo2', 'birstate', 'age'], 
                 reg=reg_form3):
        self.races = races
        self.year = year
        self.varlist = varlist
        self.reg = reg_form3
        self.df = df_dict[year]
    
    def df_race(self, j=0):
        return self.df[self.df.race == j]
        
    def models(self, df_i):
        x = smf.ols('isei ~'+self.reg, df_i).fit(cov_type='HC0')
        return x
    
    def av_char(self, j=0):
        char = [1]
        for v in self.varlist:
            if v == 'age':
                var_list = [self.df_race(j).age.mean(), np.power(self.df_race(j).age.mean(), 2)]
                char += var_list
            else:
                var_list = self.df_race(j)[v].value_counts(sort=False, normalize=True).tolist()
                char += var_list[1:]
        return pd.Series(char, index=self.models(self.df_race(j)).params.index)
    
    def chars(self, k):
        x_list = [np.array(self.av_char(j)) for j in self.races]
        return x_list[k]
        
    def betas(self, k, d='classic0'):
        bs = [np.array(self.models(self.df_race(j)).params) for j in self.races]
        if d=='ransom':
            b_param = smf.ols('isei~race+'+self.reg,self.df).fit().params.tolist()
            b_raw = [b_param[0]]+b_param[4:]
            b_c = np.array(b_raw)
        elif d=='neumark':
            b_c = np.array(smf.ols('isei~'+self.reg, self.df).fit().params)
        else:
            tau_dict = {'classic0':0, 'classic1':1, 'reimers':0.5, 
                        'cotton':self.df.race.value_counts(normalize=True).to_dict()[0]}
            tau = tau_dict[d]
            b_c = tau*bs[0] + (1-tau)*bs[1]
        b_list = bs + [b_c]
        return b_list[k]
    
    def results(self, d='classic0', p='simple'):
        R = self.betas(1,d)@self.chars(1) - self.betas(0,d)@self.chars(0)
        E = self.betas(2,d)@(self.chars(1) - self.chars(0))
        D_former = np.transpose(self.chars(1))@(self.betas(1,d)-self.betas(2,d))
        D_latter = np.transpose(self.chars(0))@(self.betas(2,d)-self.betas(0,d))
        U = D_former + D_latter
        results_list = [R,E,D_former, D_latter, U]
        results_df = pd.DataFrame(data=np.transpose([results_list,[x/R*100 for x in results_list]]),
                                  columns=['Level','Percentage'],
                                  index=['Raw','Explained',
                                         'Pro {}'.format(self.races[1]), 
                                         'Against {}'.format(self.races[0]), 'Unexplained'])
        if p=='simple':
            return results_list
        elif p=='full':
            print('{}: {} and {}, using {}'.format(self.year, self.races[0], self.races[1], d.upper()))
            return results_df

In [222]:
class Oaxaca_stats(Oaxaca):
    
    # Guided by Ma and Ng (2006)
    
    def __init__(self, year, T=10000):
        self.ob = Oaxaca(year)
        self.T = T
    
    def bootstrap(self, j=0, d='classic0'):
        df_boot = self.ob.df_race(j)
        rfit, rres = self.ob.models(df_boot).fittedvalues, self.ob.models(df_boot).resid
        data = pd.DataFrame(data=np.transpose([np.log(np.square(rres)), rfit, np.square(rfit)]), 
                            columns=['epsilon', 'fitted', 'fitted2'])
        x = smf.ols('epsilon ~ fitted + fitted2', data=data).fit()
        pick = np.random.choice(a=x.fittedvalues, size=len(x.fittedvalues), replace=True)
        array1 = np.array((np.ones(len(pick)), rfit, np.square(rfit).T, pick))
        array2 = np.concatenate((x.params.values,[1]))
        e_star = np.sign(rres.values) * np.exp(0.5*array2@array1).T
        E = rfit + e_star
        return E

In [223]:
Oaxaca_stats(1970).bootstrap(0)

0        38.168178
2        69.207284
4        23.083980
6        53.751138
11       29.267550
12       17.616539
17       54.329426
19       37.016914
27       15.716179
28       36.391043
30       40.316375
34       44.233321
37       32.389262
40       58.355093
41       71.334323
42        9.746140
43       52.048892
44       61.886317
45       53.818488
48       13.864160
49       18.102220
57       31.853679
58       38.352742
59       29.426338
60       15.869652
62       33.698762
64       40.464051
66       54.153708
67       31.736108
70       19.214157
           ...    
51779    11.942873
51780    22.605148
51782    17.059717
51785    73.971335
51787    21.056925
51788    15.380731
51790    70.650361
51791    18.603013
51793    30.820064
51794    31.988707
51795    15.778623
51796     9.448955
51798     9.613666
51802    30.754666
51803    23.175259
51804    21.502459
51807    16.859516
51808    34.228978
51809    23.045253
51812    34.997024
51813    21.258672
51814    14.

In [187]:
np.sign(-3)

-1